In [1]:
# IMPORTS

In [2]:
import sys
import time
from pathlib import Path
from geopy.geocoders import Nominatim
import pygtfs
import os
from graph_tool.all import *
from graph_tool.topology import shortest_distance, shortest_path
from pyrosm import get_data, OSM
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import shapely.geometry
from datetime import datetime, date, time, timedelta
import time as tm
from pyrosm.data import sources
from queue import Queue
import heapq
from collections import defaultdict
import graphviz
from geopy.exc import GeocoderServiceError
import math
import folium
from IPython.display import display, SVG
from math import radians, cos, sin, asin, sqrt
import pandas as pd

from aves.data import eod, census
from aves.features.utils import normalize_rows

In [3]:
# INFO GETTER

In [4]:
start = tm.time()
print("GETTING OSM INFO")

# PATHS
AVES_ROOT = Path("..")
EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"
OSM_PATH = AVES_ROOT / "data" / "external" / "OSM"

## OSM ##

def get_osm_data():
    """
    Obtains the required OpenStreetMap data using the 'pyrosm' library. This gives the map info of Santiago.

    Returns:
        graph: osm data converted to a graph
    """
    # Download latest OSM data
    fp = get_data(
        "Santiago",
        update=True,
        directory=OSM_PATH
    )

    osm = OSM(fp)

    nodes, edges = osm.get_network(nodes=True)

    graph = Graph()

    # Create vertex properties for lon and lat
    lon_prop = graph.new_vertex_property("float")
    lat_prop = graph.new_vertex_property("float")
    
    # Create properties for the ids
    # Every OSM node has its unique id, different from the one given in the graph
    node_id_prop = graph.new_vertex_property("long")
    graph_id_prop = graph.new_vertex_property("long")
    
    # Create edge properties
    u_prop = graph.new_edge_property("long")
    v_prop = graph.new_edge_property("long")
    length_prop = graph.new_edge_property("double")
    weight_prop = graph.new_edge_property("double")

    vertex_map = {}

    print("GETTING OSM NODES...")
    for index, row in nodes.iterrows():
        lon = row['lon']
        lat = row['lat']
        node_id = row['id']
        graph_id = index
        node_coords[node_id] = (lat, lon)
        
        vertex = graph.add_vertex()
        vertex_map[node_id] = vertex
        
        # Assigning node properties
        lon_prop[vertex] = lon
        lat_prop[vertex] = lat
        node_id_prop[vertex] = node_id
        graph_id_prop[vertex] = graph_id

    # Assign the properties to the graph
    graph.vertex_properties["lon"] = lon_prop
    graph.vertex_properties["lat"] = lat_prop
    graph.vertex_properties["node_id"] = node_id_prop
    graph.vertex_properties["graph_id"] = graph_id_prop
    
    print("DONE")
    print("GETTING OSM EDGES...")

    for index, row in edges.iterrows():
        source_node = row['u']
        target_node = row['v']
        
        if row["length"] < 2 or source_node == "" or target_node == "":
            continue # Skip edges with empty or missing nodes
            
        if source_node not in vertex_map or target_node not in vertex_map:
            print(f"Skipping edge with missing nodes: {source_node} -> {target_node}")
            continue  # Skip edges with missing nodes
            
        source_vertex = vertex_map[source_node]
        target_vertex = vertex_map[target_node]
        
        if not graph.vertex(source_vertex) or not graph.vertex(target_vertex):
            print(f"Skipping edge with non-existent vertices: {source_vertex} -> {target_vertex}")
            continue  # Skip edges with non-existent vertices
            
        # Calculate the distance between the nodes and use it as the weight of the edge
        source_coords = node_coords[source_node]
        target_coords = node_coords[target_node]
        distance = abs(source_coords[0] - target_coords[0]) + abs(source_coords[1] - target_coords[1])
        
        e = graph.add_edge(source_vertex, target_vertex)
        u_prop[e] = source_node
        v_prop[e] = target_node
        length_prop[e] = row["length"]
        weight_prop[e] = distance
        
    graph.edge_properties["u"] = u_prop
    graph.edge_properties["v"] = v_prop
    graph.edge_properties["length"] = length_prop
    graph.edge_properties["weight"] = weight_prop

    print("OSM DATA HAS BEEN SUCCESSFULLY RECEIVED")
    return graph

# OSM Graph
node_coords = {}
osm_graph = get_osm_data()
osm_vertices = osm_graph.vertices()

# AUX FUNCTION FOR DEBUGGING
def print_graph(graph):
    """
    Prints the vertices and edges of the graph.
    """
    print("Vertices:")
    for vertex in graph.vertices():
        print(f"Vertex ID: {int(vertex)}, lon: {graph.vertex_properties['lon'][vertex]}, lat: {graph.vertex_properties['lat'][vertex]}")

    print("\nEdges:")
    for edge in graph.edges():
        source = int(edge.source())
        target = int(edge.target())
        print(f"Edge: {source} -> {target}")

# AUX FUNCTIONS TO FIND NODES
def find_node_by_coordinates(graph, lon, lat):
    """
    Finds a node in the graph based on its coordinates (lon, lat).

    Parameters:
        graph (graph): the graph containing the node coordinates.
        lon (float): the longitude of the node.
        lat (float): the latitude of the node.

    Returns:
        vertex: the vertex in the graph with the specified coordinates, or None if not found.
    """
    for vertex in graph.vertices():
        if graph.vertex_properties["lon"][vertex] == lon and graph.vertex_properties["lat"][vertex] == lat:
            return vertex
    return None

def find_node_by_id(graph, node_id):
    """
    Finds a node in the graph based on its id.

    Parameters:
        graph (graph): the graph containing the node coordinates.
        node_id (long): the id of the node.

    Returns:
        vertex: the vertex in the graph with the specified id, or None if not found.
    """
    for vertex in graph.vertices():
        if graph.vertex_properties["node_id"][vertex] == node_id:
            return vertex
    return None

def find_nearest_node(graph, latitude, longitude):
    """
    Finds the nearest node in the graph to a given set of coordinates.

    Parameters:
        graph (graph): the graph containing the nodes.
        latitude (float): the latitude of the coordinates.
        longitude (float): the longitude of the coordinates.

    Returns:
        vertex: the vertex in the graph closest to the given coordinates.
    """
    query_point = np.array([longitude, latitude])

    # Obtains vertex properties: 'lon' and 'lat'
    lon_prop = graph.vertex_properties['lon']
    lat_prop = graph.vertex_properties['lat']

    # Calculates the euclidean distances between the node's coordinates and the consulted address's coordinates
    distances = np.linalg.norm(np.vstack((lon_prop.a, lat_prop.a)).T - query_point, axis=1)

    # Finds the nearest node's index
    nearest_node_index = np.argmin(distances)
    nearest_node = graph.vertex(nearest_node_index)

    return nearest_node

def get_largest_component(component_sizes):
    largest_size = max(component_sizes)
    largest_component = [i for i, size in enumerate(component_sizes) if size == largest_size]
    return largest_component


def analyze_connectivity(graph):
    """
    Analyzes the connectivity of the graph.

    Parameters:
        graph (graph): the graph to analyze.
    """
    # Verificar si el grafo es conexo
    is_graph_connected = graph.num_edges() == graph.num_vertices() - 1
    print("El grafo es conexo:", is_graph_connected)

    # Obtener los componentes conectados utilizando la función label_components()
    components = label_components(graph)[0]

    # Contar el número de componentes conectados
    num_components = len(set(components))
    print("Número de componentes conectados:", num_components)
    print("Componentes conectados:", set(components))

    # Obtener el tamaño de cada componente
    component_sizes = []
    for component_id in set(components):
        size = np.sum(components == component_id)
        component_sizes.append(size)

    print("Tamaño de cada componente:")
    for component_id, size in enumerate(component_sizes):
        print("Componente {}: {}".format(component_id, size))
    
    # Obtener los componentes aislados
    isolated_components = [i for i, size in enumerate(component_sizes) if size == 1]
    print("Número de componentes aislados:", len(isolated_components))
    print("Componentes aislados:", isolated_components)

    # Obtener el componente más grande
    largest_component = get_largest_component(component_sizes)
    largest_component_size = largest_component[0]
    print("Componente más grande: tamaño:", largest_component_size)

def create_node_id_mapping(graph):
    """
    Creates a mapping between the graph's vertices and their node ids.

    Parameters:
        graph (graph): the graph to create the mapping for.

    Returns:
        dict: a dictionary mapping the graph's vertices to their node ids.
    """
    node_id_mapping = {}
    node_id_prop = graph.vertex_properties["node_id"]
    for v in graph.vertices():
        node_id = node_id_prop[v]
        node_id_mapping[int(v)] = node_id
    return node_id_mapping

def create_edge_mapping(graph):
    """
    Creates a mapping between the graph's edges and their source and target nodes.

    Parameters:
        graph (graph): the graph to create the mapping for.

    Returns:
        dict: a dictionary mapping the graph's edges to their source and target nodes.
    """
    edge_mapping = {}
    node_id_prop = graph.vertex_properties["graph_id"]
    for e in graph.edges():
        source_vertex, target_vertex = e.source(), e.target()
        source_node_id = node_id_prop[source_vertex]
        target_node_id = node_id_prop[target_vertex]
        edge_index = graph.edge_index[e]
        edge_mapping[edge_index] = (source_node_id, target_node_id)
    return edge_mapping

node_mapping = create_node_id_mapping(osm_graph)

edge_mapping = create_edge_mapping(osm_graph)#678892 709089
#for edge_index, (source_node, target_node) in edge_mapping.items():
#    print("Edge {}: {} -> {}".format(edge_index, source_node, target_node))
    
def edge_count(graph, vertex):
    """
    Given a graph and a vertex, returns the degree of the vertex, which is the sum of its in-degree and out-degree.

    Parameters:
    graph (Graph): A graph object.
    vertex (int): The vertex for which to calculate the degree.

    Returns:
    int: The degree of the vertex.
    """
    v = graph.vertex(vertex)
    out_degree = v.out_degree()
    in_degree = v.in_degree()
    degree = out_degree + in_degree
    return degree

#v = 709089  # Índice del vértice deseado
#degree = edge_count(osm_graph, v)
#print("Número de aristas entrantes al vértice {}: {}".format(v, v.in_degree()))
#print("Número de aristas salientes del vértice {}: {}".format(v, v.out_degree()))
#print("Número de aristas total del nodo {}: {}".format(v, degree))

# Probando si el problema realmente son las aristas o no
def make_undirected(graph):
    """
    Given a directed graph, returns an undirected version of the graph.

    Parameters:
    graph (Graph): A directed graph. In this specific case, the osm graph.

    Returns:
    Graph: An undirected version of the graph.
    """
    undirected_graph = Graph(directed=False)
    vprop_map = graph.new_vertex_property("object")
    
    # Create vertex properties for lon and lat
    lon_prop = undirected_graph.new_vertex_property("float")
    lat_prop = undirected_graph.new_vertex_property("float")
    node_id_prop = undirected_graph.new_vertex_property("long")
    graph_id_prop = undirected_graph.new_vertex_property("long")
    
    # Create edge properties
    u_prop = undirected_graph.new_edge_property("long")
    v_prop = undirected_graph.new_edge_property("long")
    length_prop = undirected_graph.new_edge_property("double")
    weight_prop = undirected_graph.new_edge_property("double")
    
    undirected_vertex_map = {}
    
    for v in graph.vertices():
        new_v = undirected_graph.add_vertex()
        vprop_map[new_v] = v
        lon = graph.vertex_properties["lon"][v]
        lat = graph.vertex_properties["lat"][v]
        node_id = graph.vertex_properties["node_id"][v]
        graph_id = graph.vertex_properties["graph_id"][v]
        
        undirected_vertex_map[node_id] = new_v
        #print("NODO {} EN GRAFO {}".format(node_id, graph_id))
        
        # Assigning node properties
        lon_prop[new_v] = lon
        lat_prop[new_v] = lat
        node_id_prop[new_v] = node_id
        graph_id_prop[new_v] = graph_id
    
    # Assign the properties to the graph
    undirected_graph.vertex_properties["lon"] = lon_prop
    undirected_graph.vertex_properties["lat"] = lat_prop
    undirected_graph.vertex_properties["node_id"] = node_id_prop
    undirected_graph.vertex_properties["graph_id"] = graph_id_prop

    
    for e in graph.edges():
        source, target = e.source(), e.target()
        source_node = graph.edge_properties["u"][e]
        target_node = graph.edge_properties["v"][e]
        lgt = graph.edge_properties["length"][e]
        wt = graph.edge_properties["weight"][e]
        
        if lgt < 2 or source_node == "" or target_node == "":
            continue # Skip edges with empty or missing nodes
            
        if source_node not in undirected_vertex_map or target_node not in undirected_vertex_map:
            print(f"Skipping edge with missing nodes: {source_node} -> {target_node}")
            continue  # Skip edges with missing nodes

        source_vertex = undirected_vertex_map[source_node]
        target_vertex = undirected_vertex_map[target_node]

        if not undirected_graph.vertex(source_vertex) or not undirected_graph.vertex(target_vertex):
            print(f"Skipping edge with non-existent vertices: {source_vertex} -> {target_vertex}")
            continue  # Skip edges with non-existent vertices
            
        e = undirected_graph.add_edge(source_vertex, target_vertex)
        u_prop[e] = source_node
        v_prop[e] = target_node
        length_prop[e] = lgt
        weight_prop[e] = wt
    
    undirected_graph.edge_properties["u"] = u_prop
    undirected_graph.edge_properties["v"] = v_prop
    undirected_graph.edge_properties["length"] = length_prop
    undirected_graph.edge_properties["weight"] = weight_prop
        
    return undirected_graph

# Convertir el grafo en no dirigido
undirected_graph = make_undirected(osm_graph)

# Finds the given address in the OSM graph
def address_locator(graph, loc):
    """
    Finds the given address in the OSM graph.

    Parameters:
        graph (Graph): The OSM graph.
        loc (str): The address to be located.

    Returns:
        int: The ID of the nearest vertex in the graph.

    Raises:
        GeocoderServiceError: If there is an error with the geocoding service.

    """
    geolocator = Nominatim(user_agent="ayatori")
    while True:
        try:
            location = geolocator.geocode(loc)
            break
        except GeocoderServiceError:
            i = 0
            if i < 15:
                print("Geocoding service error. Retrying in 5 seconds...")
                tm.sleep(5)
                i+=1
            else:
                msg = "Error: Too many retries. Geocoding service may be down. Please try again later."
                print(msg)
                return
    if location is not None:
        long, lati = location.longitude, location.latitude
        nearest = find_nearest_node(graph,lati,long)
        near_lon, near_lat = graph.vertex_properties["lon"][nearest], graph.vertex_properties["lat"][nearest]
        near_location = geolocator.reverse((near_lat,near_lon))
        near_id = graph.vertex_properties["node_id"][nearest]
        graph_id = graph.vertex_properties["graph_id"][nearest]
        #print("Ubicación entregada: {}".format(loc))
        #print("Las coordenadas de la ubicación entregada son ({},{})".format(long,lati))
        #print("El vértice más cercano a la ubicación entregada está en las coordenadas ({},{})".format(near_lon, near_lat))
        #print("Dirección: {}".format(near_location))
        #print("El id del nodo es {}".format(near_id))
        #print("El id en el grafo es {}".format(graph_id))
        return nearest
    msg = "Error: Address couldn't be found."
    print(msg)

end = tm.time()
exec_time = round((end-start) / 60,3)
print("OSM INFO IS READY. EXECUTION TIME: {} MINUTES".format(exec_time)) 

GETTING OSM INFO
Downloaded Protobuf data 'Santiago.osm.pbf' (19.66 MB) to:
'/home/lysorek/aves/data/external/OSM/Santiago.osm.pbf'
GETTING OSM NODES...
DONE
GETTING OSM EDGES...
OSM DATA HAS BEEN SUCCESSFULLY RECEIVED
OSM INFO IS READY. EXECUTION TIME: 3.632 MINUTES


In [5]:
a = address_locator(undirected_graph, "aasdas")

Error: Address couldn't be found.


In [6]:
start = tm.time()
print("GETTING GTFS INFO")

## GTFS ##

def get_gtfs_data():
    """
    Reads the GTFS data from a file and creates a directed graph with its info, using the 'pygtfs' library. This gives
    the transit feed data of Santiago's public transport, including "Red Metropolitana de Movilidad" (previously known
    as Transantiago), "Metro de Santiago", "EFE Trenes de Chile", and "Buses de Acercamiento Aeropuerto".
    
    Returns:
        graphs: GTFS data converted to a dictionary of graphs, one per route.
        route_stops: Dictionary containing the stops for each route.
        special_dates: List of special calendar dates.
    """
    # Create a new schedule object using a GTFS file
    sched = pygtfs.Schedule(":memory:")    
    pygtfs.append_feed(sched, "gtfs.zip") # This takes around 2 minutes (01:51.44)
    
    # Get special calendar dates
    special_dates = []
    for cal_date in sched.service_exceptions: # Calendar_dates is renamed in pygtfs
        special_dates.append(cal_date.date.strftime("%d/%m/%Y"))

    # Create a graph per route
    graphs = {}
    stop_id_map = {}  # To assign unique ids to every stop
    stop_coords = {}
    route_stops = {}
    for route in sched.routes:
        graph = Graph(directed=True)
        stop_ids = set()
        trips = [trip for trip in sched.trips if trip.route_id == route.route_id]

        weight_prop = graph.new_edge_property("int")  # Propiedad para almacenar los pesos de las aristas

        for trip in trips:
            stop_times = trip.stop_times

            # Get the orientation of the trip
            orientation = trip.trip_id.split("-")[1]

            for i in range(len(stop_times)):
                stop_id = stop_times[i].stop_id
                sequence = stop_times[i].stop_sequence 

                if stop_id not in stop_id_map:
                    vertex = graph.add_vertex()  # Añadir un vértice vacío
                    stop_id_map[stop_id] = vertex  # Asignar el vértice al identificador de parada
                else:
                    vertex = stop_id_map[stop_id]  # Obtener el vértice existente

                stop_ids.add(vertex)

                if i < len(stop_times) - 1:
                    next_stop_id = stop_times[i + 1].stop_id

                    if next_stop_id not in stop_id_map:
                        next_vertex = graph.add_vertex()  # Añadir un vértice vacío para la siguiente parada
                        stop_id_map[next_stop_id] = next_vertex  # Asignar el vértice al identificador de parada
                    else:
                        next_vertex = stop_id_map[next_stop_id]  # Obtener el vértice existente para la siguiente parada

                    e = graph.add_edge(vertex, next_vertex)  # Añadir una arista entre las paradas
                    weight_prop[e] = 1  # Asignar peso 1 a la arista
                    
                    # Store the coordinates of each stop for this route
                    if route.route_id not in stop_coords:
                        stop_coords[route.route_id] = {}
                    if stop_id not in stop_coords[route.route_id]:
                        stop = sched.stops_by_id(stop_id)[0]
                        stop_coords[route.route_id][stop_id] = (stop.stop_lon, stop.stop_lat)
                        
                        # Store the sequence of each stop for this route
                        if route.route_id not in route_stops:
                            route_stops[route.route_id] = {}
                        route_stops[route.route_id][stop_id] = {
                            "route_id": route.route_id,
                            "stop_id": stop_id,
                            "coordinates": stop_coords[route.route_id][stop_id],
                            "visited_on_round_trip": True if orientation == "I" else False,
                            "visited_on_return_trip": True if orientation == "R" else False,
                            "sequence": sequence,
                            "arrival_times": []
                        }
                        
                # Get the arrival time for the current stop
                arrival_time = (datetime.min + stop_times[i].arrival_time).time()

                # Check if the stop ID is already in the dictionary
                if stop_id in route_stops[route.route_id]:
                    # If the stop ID is already in the dictionary, append the arrival time
                    route_stops[route.route_id][stop_id]["arrival_times"].append(arrival_time)
                #else:
                    # If the stop ID is not in the dictionary, create a new entry with the arrival time
                #    route_stops[route.route_id][stop_id] = {
                #        "route_id": route.route_id,
                #        "stop_id": stop_id,
                #        "coordinates": stop_coords[route.route_id][stop_id],
                #        "visited_on_round_trip": True if orientation == "I" else False,
                #        "visited_on_return_trip": True if orientation == "R" else False,
                #        "sequence": sequence,
                #        "arrival_times": [arrival_time]
                #    }

        graphs[route.route_id] = graph
        # Group the stops by direction to get the stops visited on the round trip and the return trip
        stops_by_direction = {"round_trip": [], "return_trip": []}
        for trip in trips:
            stop_times = trip.stop_times
            stops = [stop_times[i].stop_id for i in range(len(stop_times))]
            
            # Determine the direction of the trip
            if trip.direction_id == 0:
                stops_by_direction["round_trip"].extend(stops)
            else:
                stops_by_direction["return_trip"].extend(stops)


        # Get the unique stops visited on the round trip and the return trip
        round_trip_stops = set(stops_by_direction["round_trip"])
        return_trip_stops = set(stops_by_direction["return_trip"])
        
        for stop_id in round_trip_stops:
            if stop_id in stop_coords[route.route_id]:
                if stop_id in route_stops[route.route_id]:
                    route_stops[route.route_id][stop_id]["visited_on_return_trip"] = True
                else:
                    route_stops[route.route_id][stop_id] = {
                        "route_id": route.route_id,
                        "stop_id": stop_id,
                        "coordinates": stop_coords[route.route_id][stop_id],
                        "visited_on_round_trip": True,
                        "visited_on_return_trip": False,
                        "sequence": sequence,
                        "arrival_times": []
                    }
        for stop_id in return_trip_stops:
            if stop_id in stop_coords[route.route_id]:
                if stop_id in route_stops[route.route_id]:
                    route_stops[route.route_id][stop_id]["visited_on_return_trip"] = True
                else:
                    route_stops[route.route_id][stop_id] = {
                        "route_id": route.route_id,
                        "stop_id": stop_id,
                        "coordinates": stop_coords[route.route_id][stop_id],
                        "visited_on_round_trip": False,
                        "visited_on_return_trip": True,
                        "sequence": sequence,
                        "arrival_times": []
                    }

    print("DONE")
    print("STORING ROUTE GRAPHS...")

    # Store graphs into a file
    for route_id, graph in graphs.items():
        weight_prop = graph.new_edge_property("int")  # Crear una nueva propiedad de peso de arista

        for e in graph.edges():  # Iterar sobre las aristas del grafo
            weight_prop[e] = 1  # Asignar el peso 1 a cada arista

        graph.edge_properties["weight"] = weight_prop  # Asignar la propiedad de peso al grafo
        
        data_dir = "gtfs_routes"
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)
        graph.save(f"{data_dir}/{route_id}.gt")
    
    print("GTFS DATA RECEIVED SUCCESSFULLY")
    return graphs, route_stops, special_dates

# GTFS Graph
gtfs_graph, route_stops, special_dates = get_gtfs_data()

def get_route_coordinates():
    """
    Prompts the user to enter a route ID and returns a list of the coordinates of each stop for that route.
    
    Returns:
        stop_coords_list: List of tuples containing the coordinates of each stop for the specified route.
    """
    route_example = "506"
    while True:
        route_id = input(
            "Ingresa recorrido (Ejemplo: '506'. Presiona Enter para usarlo): ") or route_example
        if route_id.strip() != '':
            print("Ruta ingresada: " + route_id)
            break
    stop_coords_list = [coord for stop_id, coord in stop_coords[route_id].items()]
    return stop_coords_list

end = tm.time()
exec_time = round((end-start) / 60,3)
print("GTFS INFO IS READY. EXECUTION TIME: {} MINUTES".format(exec_time)) 

GETTING GTFS INFO
Loading GTFS data for <class 'pygtfs.gtfs_entities.Agency'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Stop'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Transfer'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Route'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Fare'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.FareRule'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.ShapePoint'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Service'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.ServiceException'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Trip'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Frequency'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.StopTime'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.FeedInfo'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Translation'>:
4 records read for <class 'pygtfs.gtfs_entities.Agency'>.
..11638 records read for <class 'pygtfs.gtfs_e

In [7]:
# MAPPING FUNCTIONS

In [8]:
# Define the Haversine formula for calculating distances between two points
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points on the earth (specified in decimal degrees).
    
    Parameters:
    lon1 (float): Longitude of the first point in decimal degrees.
    lat1 (float): Latitude of the first point in decimal degrees.
    lon2 (float): Longitude of the second point in decimal degrees.
    lat2 (float): Latitude of the second point in decimal degrees.
    
    Returns:
    float: The distance between the two points in kilometers.
    """
    R = 6372.8  # Earth radius in kilometers
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat / 2)**2 + cos(lat1) * cos(lat2) * sin(dLon / 2)**2
    c = 2 * asin(sqrt(a))
    return R * c

def find_next_stop(route_stops, coords):
    """
    Find the nearest stop to the given coordinates among the stops in the route_stops dictionary.
    
    Parameters:
    route_stops (dict): A dictionary containing the stops for each route.
    coords (tuple): The coordinates (latitude, longitude) of the current location.
    
    Returns:
    tuple: The coordinates (latitude, longitude) of the nearest stop.
    """
    min_distance = float("inf")
    nearest_stop_coords = None
    for route_id, stops in route_stops.items():
        for stop_info in stops.values():
            stop_coords = stop_info["coordinates"]
            distance = haversine(coords[1], coords[0], stop_coords[1], stop_coords[0])
            if distance < min_distance and stop_coords != coords:
                min_distance = distance
                next_stop_coords = stop_coords
    return next_stop_coords

map_colors= ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 
         'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue',
         'lightgreen', 'gray', 'black', 'lightgray']

def map_route_stops(route_list, color_list, stops_flag):
    """
    Create a map showing the stops visited on the round trip for the specified routes.
    
    Parameters:
    route_list (list): A list of route IDs.
    color_list (list): A list of colors for each route.
    stops_flag (bool): A flag indicating whether to display the stops on the map.
    
    Returns:
    folium.Map: A map object showing the stops and routes.
    """
    # Map the stops visited on the round trip
    map = folium.Map(location=[-33.45, -70.65], zoom_start=12)
    
    color_id = 0
    for route_id in route_list:
        # Get the stops for the specified route
        stops = route_stops.get(route_id, {})

        # Filter the stops that are visited on the round trip
        round_trip_stops = [stop_info for stop_info in stops.values() if stop_info["visited_on_round_trip"]]

        # Sort the stops by their sequence number in the trip
        round_trip_stops.sort(key=lambda stop: stop["sequence"])
        for stop in round_trip_stops:
            print(stop['sequence'])

        if stops_flag:
            for stop_info in round_trip_stops:
                folium.Marker(location=[stop_info["coordinates"][1], stop_info["coordinates"][0]], popup=stop_info["stop_id"],
                               icon=folium.Icon(color='lightgray', icon='minus')).add_to(map)

        folium.PolyLine(locations=[[stop_info["coordinates"][1], stop_info["coordinates"][0]] for stop_info in round_trip_stops],
                        color=map_colors[color_id], weight=2).add_to(map)
        
        color_id+=1

    return map

def is_between_coordinates(to_check_coords, coordinates1, coordinates2):
    """
    Check if the given coordinates are between the two other sets of coordinates.
    
    Parameters:
    to_check_coords (tuple): The coordinates to check.
    coordinates1 (tuple): The first set of coordinates.
    coordinates2 (tuple): The second set of coordinates.
    
    Returns:
    bool: True if the to_check_coords are between the other two sets of coordinates, False otherwise.
    """
    # Check if the to_check_coords are between coordinates1 and coordinates2
    lon1, lat1 = coordinates1
    lon2, lat2 = coordinates2
    lon, lat = to_check_coords
    if (lon1 <= lon <= lon2 or lon2 <= lon <= lon1) and (lat1 <= lat <= lat2 or lat2 <= lat <= lat1):
        return True
    return False

def map_partial_route_stops(route_list, color_list, stops_flag, address1, address2):
    """
    Given a list of routes, a list of colors, a flag to indicate whether to show stops or not, and two addresses,
    returns a folium map showing the stops of the specified routes that are visited on the round trip and are between
    the two addresses.

    Parameters:
    route_list (list): A list of route IDs.
    color_list (list): A list of colors to use for the routes.
    stops_flag (bool): A flag to indicate whether to show stops or not.
    address1 (str): The starting address.
    address2 (str): The ending address.

    Returns:
    folium.Map: A folium map showing the stops of the specified routes that are visited on the round trip and are between
    the two addresses.
    """
    source = address_locator(undirected_graph, address1)
    target = address_locator(undirected_graph, address2)
    coordinates1 = (undirected_graph.vertex_properties["lon"][source], undirected_graph.vertex_properties["lat"][source])
    coordinates2 = (undirected_graph.vertex_properties["lon"][target], undirected_graph.vertex_properties["lat"][target])
    map = folium.Map(location=[-33.45, -70.65], zoom_start=12)
    color_id = 0
    for route_id in route_list:
        # Get the stops for the specified route
        stops = route_stops.get(route_id, {})
        stops.sort(key=lambda stop: stop["sequence"])

        # Filter the stops that are visited on the round trip and are between coordinates1 and coordinates2
        #round_trip_stops = [stop_info for stop_info in stops.values() if stop_info["visited_on_round_trip"] and
        #                    is_between_coordinates(stop_info["coordinates"], coordinates1, coordinates2)]
        round_trip_stops = [stop_info for stop_info in stops.values() if stop_info["visited_on_round_trip"]]

        # Sort the stops by their coordinates
        #round_trip_stops.sort(key=lambda stop: stop["sequence"])

        if stops_flag:
            for stop_info in round_trip_stops:
                folium.Marker(location=[stop_info["coordinates"][1], stop_info["coordinates"][0]], popup=stop_info["stop_id"],
                               icon=folium.Icon(color='lightgray', icon='minus')).add_to(map)

        folium.PolyLine(locations=[[stop_info["coordinates"][1], stop_info["coordinates"][0]] for stop_info in round_trip_stops],
                        color=map_colors[color_id], weight=2).add_to(map)
        
        color_id+=1
        #folium.PolyLine(locations=[[find_next_stop(route_stops, (stop_info["coordinates"][1], stop_info["coordinates"][0]))]
        #                           for stop in round_trip_stops], color='red', weight=2).add_to(map)
    return map


In [62]:
def get_stop_coords(route_stops, stop_id):
    """
    Given a dictionary of route stops and a stop ID, returns the coordinates of the stop with the given ID.
    If the stop ID is not found, returns None.

    Parameters:
    route_stops (dict): A dictionary of route stops.
    stop_id (int): The ID of the stop to get the coordinates for.

    Returns:
    tuple: A tuple of two floats representing the longitude and latitude of the stop with the given ID.
    None: If the stop ID is not found.
    """
    for route_id, stops in route_stops.items():
        for stop_info in stops.values():
            if stop_info["stop_id"] == stop_id:
                return stop_info["coordinates"]
    return None

def get_stop_ids(route_stops, coords, margin):
    """
    Given a dictionary of route stops, a tuple of coordinates, and a margin, returns a list of stop IDs
    that are within the specified margin of the given coordinates, along with their orientations.

    Parameters:
    route_stops (dict): A dictionary of route stops.
    coords (tuple): A tuple of two floats representing the longitude and latitude of the coordinates to search around.
    margin (float): The maximum distance (in kilometers) from the given coordinates to include stops in the result.

    Returns:
    tuple: A tuple of two lists. The first list contains the stop IDs that are within the specified margin of the given coordinates.
    The second list contains tuples of stop IDs and their orientations.
    """
    stop_ids = []
    orientations = []
    for route_id, stops in route_stops.items():
        for stop_info in stops.values():
            stop_coords = stop_info["coordinates"]
            distance = haversine(coords[1], coords[0], stop_coords[1], stop_coords[0])
            if distance <= margin:
                orientation = "Round trip?: {}".format(stop_info["visited_on_round_trip"])
                stop_ids.append(stop_info["stop_id"])
                orientations.append((stop_info["stop_id"],orientation))
    return stop_ids, orientations

def find_nearest_stops(address, margin):
    """
    Given an address and a margin, returns a list of the nearest stop IDs and their orientations.

    Parameters:
    address (str): The address to search around.
    margin (float): The maximum distance (in kilometers) from the given address to include stops in the result.

    Returns:
    tuple: A tuple of two lists. The first list contains the stop IDs that are within the specified margin of the given address.
    The second list contains tuples of stop IDs and their orientations.
    """
    v = address_locator(undirected_graph, str(address))
    v_lon = undirected_graph.vertex_properties['lon'][v]
    v_lat = undirected_graph.vertex_properties['lat'][v]
    v_coords = (v_lon, v_lat)
    nearest_stops, orientations = get_stop_ids(route_stops, v_coords, margin)
    return nearest_stops, orientations

def find_closest_stop(coordinates, stop_ids):
    min_distance = float("inf")
    closest_stop_id = None
    for stop_id in stop_ids:
        stop_coords = get_stop_coords(route_stops, stop_id)
        distance = haversine(coordinates[1], coordinates[0], stop_coords[0], stop_coords[1])
        if distance < min_distance:
            min_distance = distance
            closest_stop_id = stop_id
    return closest_stop_id

def get_service_stops(route_stops, route_id):
    """
    Given a dictionary of route stops and a route ID, returns a list of stop IDs for the stops on the given route.

    Parameters:
    route_stops (dict): A dictionary of route stops.
    route_id (int): The ID of the route to get the stops for.

    Returns:
    list: A list of stop IDs for the stops on the given route.
    """
    stops = route_stops.get(route_id, {})
    return stops.keys()

def route_stop_matcher(route_stops, route_id, stop_id):
    """
    Given a dictionary of route stops, a route ID, and a stop ID, returns True if the stop ID is on the given route,
    and False otherwise.

    Parameters:
    route_stops (dict): A dictionary of route stops.
    route_id (int): The ID of the route to check.
    stop_id (int): The ID of the stop to check.

    Returns:
    bool: True if the stop ID is on the given route, False otherwise.
    """
    stop_list = get_service_stops(route_stops, route_id)
    return (stop_id in stop_list)

def is_route_near_coordinates(route_stops, route_id, coordinates, margin):
    """
    Given a dictionary of route stops, a route ID, a tuple of coordinates, and a margin, returns True if the route
    has a stop within the specified margin of the given coordinates, and False otherwise.

    Parameters:
    route_stops (dict): A dictionary of route stops.
    route_id (int): The ID of the route to check.
    coordinates (tuple): A tuple of two floats representing the longitude and latitude of the coordinates to search around.
    margin (float): The maximum distance (in kilometers) from the given coordinates to include stops in the result.

    Returns:
    bool: True if the route has a stop within the specified margin of the given coordinates, False otherwise.
    """
    for stop_info in route_stops[route_id].values():
        #print(stop_info["stop_id"])
        stop_coords = stop_info["coordinates"]
        distance = haversine(coordinates[1], coordinates[0], stop_coords[1], stop_coords[0])
        if distance <= margin:
            return route_id
    return False

#coords = (-70.7592749,-33.4708183)
#margin = 0.1
#is_route_near_coordinates(route_stops, "404", coords, margin)

def is_route_near_both_coordinates(route_stops, route_id, coordinates1, coordinates2, margin):
    """
    Determines if a route is near both given coordinates within a certain margin.

    Parameters:
    route_stops (dict): A dictionary containing information about stops on a route.
    route_id (str): The ID of the route to check.
    coordinates1 (tuple): A tuple containing the latitude and longitude of the first coordinate.
    coordinates2 (tuple): A tuple containing the latitude and longitude of the second coordinate.
    margin (float): The maximum distance in kilometers that a route can be from both coordinates.

    Returns:
    bool: True if the route is near both coordinates, False otherwise.
    """
    stops_near_coordinates1 = set()
    stops_near_coordinates2 = set()
    for stop_info in route_stops[route_id].values():
        stop_coords = stop_info["coordinates"]
        distance1 = haversine(coordinates1[1], coordinates1[0], stop_coords[1], stop_coords[0])
        distance2 = haversine(coordinates2[1], coordinates2[0], stop_coords[1], stop_coords[0])
        if distance1 <= margin:
            stops_near_coordinates1.add(stop_info["stop_id"])
        if distance2 <= margin:
            stops_near_coordinates2.add(stop_info["stop_id"])
    return bool(stops_near_coordinates1.intersection(stops_near_coordinates2))      

def route_checker(route_stops, address1, address2, margin):
    """
    Finds all routes that are near both given addresses within a certain margin.

    Parameters:
    route_stops (dict): A dictionary containing information about stops on a route.
    address1 (str): The first address to check.
    address2 (str): The second address to check.
    margin (float): The maximum distance in kilometers that a route can be from both addresses.

    Returns:
    list: A list of route IDs that are near both addresses.
    """
    source = address_locator(undirected_graph, address1)
    target = address_locator(undirected_graph, address2)
    
    coordinates1 = (undirected_graph.vertex_properties["lon"][source], undirected_graph.vertex_properties["lat"][source])
    coordinates2 = (undirected_graph.vertex_properties["lon"][target], undirected_graph.vertex_properties["lat"][target])

    valid_services=[]
    for route_id, stops in route_stops.items():
        rid = route_id
        if is_route_near_both_coordinates(route_stops, rid, coordinates1, coordinates2, margin):
            valid_services.append(rid)
    display(map_partial_route_stops(valid_services, map_colors, 0,address1, address2))
    return valid_services

def get_bus_orientation(route_id, stop_id):
    stop_times = pd.read_csv("stop_times.txt")
    filtered_stop_times = stop_times[(stop_times["trip_id"].str.startswith(route_id)) & (stop_times["stop_id"] == stop_id)]
    
    for trip_id in filtered_stop_times["trip_id"]:
        orientation = trip_id.split("-")[1]
        if orientation == "I":
            return "round"
        elif orientation == "R":
            return "return"
    
    return None

def connection_finder(route_stops, stop_id_1, stop_id_2):
    """
    Finds all routes that have stops at both given stop IDs.

    Parameters:
    route_stops (dict): A dictionary containing information about stops on a route.
    stop_id_1 (str): The ID of the first stop to check.
    stop_id_2 (str): The ID of the second stop to check.

    Returns:
    list: A list of route IDs that have stops at both given stop IDs.
    """
    connected_routes = []
    for route_id, stops in route_stops.items():
        stop_ids = [stop_info["stop_id"] for stop_info in stops.values()]
        #source_stop_orientation = get_bus_orientation(route_id, stop_id_1)
        #target_stop_orientation = get_bus_orientation(route_id, stop_id_2)
        #if stop_id_1 in stop_ids and stop_id_2 in stop_ids and source_stop_orientation == target_stop_orientation:
        if stop_id_1 in stop_ids and stop_id_2 in stop_ids:
            connected_routes.append(route_id)
    return connected_routes

def get_routes_at_stop(route_stops, stop_id):
    """
    Finds all routes that have a stop at the given stop ID.

    Parameters:
    route_stops (dict): A dictionary containing information about stops on a route.
    stop_id (str): The ID of the stop to check.

    Returns:
    list: A list of route IDs that have a stop at the given stop ID.
    """
    routes = [route_id for route_id in route_stops.keys() if stop_id in get_service_stops(route_stops, route_id) and connection_finder(route_stops, stop_id, stop_id)]
    return routes

def check_night_routes(valid_services, is_nighttime):
    """
    Filters the given list of route IDs to only include night routes if is_nighttime is True.

    Parameters:
    valid_services (list): A list of route IDs to filter.
    is_nighttime (bool): True if it is nighttime, False otherwise.

    Returns:
    list: A list of route IDs that are night routes if is_nighttime is True, or all route IDs otherwise.
    """
    if is_nighttime:
        nighttime_routes = [route_id for route_id in valid_services if route_id.endswith("N")]
        if nighttime_routes:
            return nighttime_routes
        else:
            return None
    else:
        daytime_routes = [route_id for route_id in valid_services if not route_id.endswith("N")]
        if daytime_routes:
            return daytime_routes
        else:
            return None

def is_nighttime(source_hour):
    """
    Determines if the given hour is during the nighttime.

    Parameters:
    source_hour (datetime.time): The hour to check.

    Returns:
    bool: True if the hour is during the nighttime, False otherwise.
    """
    start_time = time(0, 0, 0)
    end_time = time(5, 30, 0)
    #source_time = datetime.strptime(source_hour, "%H:%M:%S").time()
    if start_time <= source_hour <= end_time:
        return True
    else:
        return False
    
def check_express_routes(valid_services, is_rush_hour):
    """
    Filters the given list of route IDs to only include express routes if is_rush_hour is True.

    Parameters:
    valid_services (list): A list of route IDs to filter.
    is_rush_hour (bool): True if it is rush hour, False otherwise.

    Returns:
    list: A list of route IDs that are express routes if is_rush_hour is True, or all route IDs otherwise.
    """
    if is_rush_hour:
        return valid_services
    else:
        regular_hour_routes = [route_id for route_id in valid_services if not route_id.endswith("e")]
        return regular_hour_routes
    
def is_rush_hour(source_hour):
    """
    Determines if the given hour is during rush hour.

    Parameters:
    source_hour (datetime.time): The hour to check.

    Returns:
    bool: True if the hour is during rush hour, False otherwise.
    """
    am_start_time = time(5, 30, 0)
    am_end_time = time(9, 0, 0)
    pm_start_time = time(17, 30, 0)
    pm_end_time = time(21, 0, 0)
    #source_time = datetime.strptime(source_hour, "%H:%M:%S").time()
    if am_start_time <= source_hour <= am_end_time or pm_start_time <= source_hour <= pm_end_time:
        return True
    else:
        return False
    
def is_holiday(date_string):
    """
    Checks if a given date is a holiday.

    Parameters:
    date_string (str): A string representing the date in the format "dd/mm/yyyy".

    Returns:
    bool: True if the date is a holiday, False otherwise.
    """
    # Local holidays
    if date_string in special_dates:
        return True
    date_obj = datetime.strptime(date_string, "%d/%m/%Y")

    # Weekend days
    day_of_week = date_obj.weekday()
    if day_of_week == 5 or day_of_week == 6:
        return True
    return False


#print(special_dates)
#is_holiday("21/06/2023")

#find_nearest_connections(route_stops, "Beauchef 850, Santiago", "Av. Las Parcelas 3052, Maipu")

#get_routes_at_stop(route_stops, "PI263")

#connection_finder(route_stops, "PI1412", "PI1552")
#print(list(route_stops.items())[:1])

#find_nearest_stops("Boriquen 146, Maipu", 0.2)
#RUNNING THIS, I GET THIS
#[('PI1597', 'Round trip?: True'),
# ('PI213', 'Round trip?: True'),
# ('PI240', 'Round trip?: False'),
# ('PI263', 'Round trip?: False'),
# ('PI263', 'Round trip?: False'),
# ('PI1597', 'Round trip?: True'),
# ('PI213', 'Round trip?: True'),
# ('PI240', 'Round trip?: False'),
# ('PI263', 'Round trip?: False'),
# ('PI1597', 'Round trip?: True'),
# ('PI213', 'Round trip?: True'),
# ('PI240', 'Round trip?: False'),
# ('PI1597', 'Round trip?: True'),
# ('PI213', 'Round trip?: True'),
# ('PI1597', 'Round trip?: True'),
# ('PI213', 'Round trip?: True'),
# ('PI263', 'Round trip?: True')]
#connection_finder(route_stops,"PI1412","PI777")

def get_trip_day_suffix(date):
    # Get the day of the week
    date_object = datetime.strptime(date, "%d/%m/%Y")
    day_of_week = date_object.weekday()

    # Determine the trip ID suffix based on the day of the week
    if day_of_week < 5:
        trip_day_suffix = "L"
    elif day_of_week == 5:
        trip_day_suffix = "S"
    else:
        trip_day_suffix = "D"

    return trip_day_suffix

def get_arrival_times(route_id, stop_id, source_date):
    """
    Returns the arrival times for a given route and stop.

    Parameters:
    route_id (str): A string representing the ID of the route.
    stop_id (str): A string representing the ID of the stop.

    Returns:
    tuple: A tuple containing a string representing the bus orientation ("round" or "return") and a list of datetime objects representing the arrival times.
    """
    # Read the frequencies.txt file
    frequencies = pd.read_csv("frequencies.txt")

    # Filter the frequencies for the given route ID
    route_frequencies = frequencies[frequencies["trip_id"].str.startswith(route_id)]
    
    # Get the day suffix
    day_suffix = get_trip_day_suffix(source_date)

    # Get the arrival times for the stop for each trip
    stop_route_times = []
    bus_orientation = ""
    for _, row in route_frequencies.iterrows():
        start_time = pd.Timestamp(row["start_time"])
        if row["end_time"] == "24:00:00":
            end_time = pd.Timestamp("23:59:59")
        else:
            end_time = pd.Timestamp(row["end_time"])
        headway_secs = row["headway_secs"]
        round_trip_id = f"{route_id}-I-{day_suffix}"
        return_trip_id = f"{route_id}-R-{day_suffix}"
        round_stop_times = pd.read_csv("stop_times.txt").query(f"trip_id.str.startswith('{round_trip_id}') and stop_id == '{stop_id}'")
        return_stop_times = pd.read_csv("stop_times.txt").query(f"trip_id.str.startswith('{return_trip_id}') and stop_id == '{stop_id}'")
        if len(round_stop_times) == 0 and len(return_stop_times) == 0:
            return
        elif len(round_stop_times) > 0:
            bus_orientation = "round"
            stop_time = pd.Timestamp(round_stop_times.iloc[0]["arrival_time"])
        elif len(return_stop_times) > 0:
            bus_orientation = "return"
            stop_time = pd.Timestamp(return_stop_times.iloc[0]["arrival_time"])
        for freq_time in pd.date_range(start_time, end_time, freq=f"{headway_secs}s"):
            freq_time_str = freq_time.strftime("%H:%M:%S")
            freq_time = datetime.strptime(freq_time_str, "%H:%M:%S")
            stop_route_time = datetime.combine(datetime.min, stop_time.time()) + timedelta(seconds=(freq_time - datetime.min).seconds)
            if stop_route_time not in stop_route_times:
                stop_route_times.append(stop_route_time)
            stop_time += pd.Timedelta(seconds=headway_secs)

    return bus_orientation, stop_route_times

def get_time_until_next_bus(arrival_times, source_hour):
    """
    Returns the time until the next three buses.

    Parameters:
    arrival_times (list): A list of datetime objects representing the arrival times of the buses.

    Returns:
    list: A list of tuples representing the time until the next three buses in minutes and seconds.
    """
    arrival_times_today = [time for time in arrival_times if time.time() >= source_hour]
    if len(arrival_times_today) == 0:
        return None
    else:
        # Sort the arrival times in ascending order
        arrival_times_today.sort()

        # Get the datetime objects for the next three buses
        next_buses = []
        for i in range(min(3, len(arrival_times_today))):
            next_arrival_time = arrival_times_today[i]
            next_bus = datetime.combine(datetime.today(), next_arrival_time.time())
            next_buses.append(next_bus)
        
        if next_buses is None:
            print("No buses today.")
        else:
            # Calculate the time until the next three buses
            time_until_next_buses = []
            for next_bus in next_buses:
                time_until_next_bus = (next_bus - datetime.combine(datetime.today(), source_hour)).total_seconds()
                minutes, seconds = divmod(time_until_next_bus, 60)
                time_until_next_buses.append((int(minutes), int(seconds)))
            
            return time_until_next_buses

def timedelta_to_hhmm(td):
    total_seconds = int(td.total_seconds())
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    return f"{hours:02d}:{minutes:02d}"
    
def timedelta_separator(td):
    total_seconds = td.total_seconds()
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)
    return minutes, seconds
    
    
def get_travel_time(trip_id, stop_ids):
    """
    Returns the travel time between two stops for a given trip.

    Parameters:
    trip_id (str): A string representing the ID of the trip.
    stop_ids (list): A list of two strings representing the IDs of the stops.

    Returns:
    datetime.timedelta: A timedelta object representing the travel time.
    """
    stop_times = pd.read_csv("stop_times.txt").query(f"trip_id.str.startswith('{trip_id}') and stop_id in {stop_ids}")
    if len(stop_times) < 2:
        return None
    arrival_times = [datetime.strptime(arrival_time, "%H:%M:%S") for arrival_time in stop_times["arrival_time"]]
    travel_time = arrival_times[1] - arrival_times[0]
    return travel_time

#get_travel_time("401-I-L", ["PI400", "PA338"]) # 00:37:35 #

a = get_bus_orientation("I08", "PI1413")
print(a)

#stop_times = pd.read_csv("stop_times.txt")
#filtered_stop_times = stop_times[stop_times["stop_id"] == "PI1412"]
#print(filtered_stop_times["trip_id"].unique())

return


In [63]:
# Define the function to set the optimal zoom level for the map
def fit_bounds(points, m):
    """
    Fits the map bounds to a given set of points.

    Parameters:
    points (list): A list of points in the format [(lat1, lon1), (lat2, lon2), ...].
    m (folium.Map): A folium map object.
    """
    df = pd.DataFrame(points).rename(columns={0:'Lat', 1:'Lon'})[['Lat', 'Lon']]
    sw = df[['Lat', 'Lon']].min().values.tolist()
    ne = df[['Lat', 'Lon']].max().values.tolist()
    m.fit_bounds([sw, ne])

# Define the function to create a map that shows the correct public transport services to take from a source to a target
def create_transport_map(route_stops, selected_path, source_date, source_hour, margin):
    """
    Creates a map that shows the correct public transport services to take from a source to a target.

    Parameters:
    route_stops (dict): A dictionary that maps route IDs to a list of stop IDs.
    selected_path (list): A list of points in the format [(lat1, lon1), (lat2, lon2), ...].
    nighttime_flag (bool): A flag indicating whether to include nighttime routes.
    rush_hour_flag (bool): A flag indicating whether to include express routes during rush hour.
    margin (float): The margin in kilometers around the given addresses.

    Returns:
    folium.Map: A folium map object.
    """
    # Note: The margin represents the kilometers around the given addresses.
    # Example: a margin of 0.1 represents 0.1 km, or 100 meters.

    geolocator = Nominatim(user_agent="ayatori")
    source_lat = selected_path[0][0]
    source_lon = selected_path[0][1]
    target_lat = selected_path[-1][0]
    target_lon = selected_path[-1][1]
    source = geolocator.reverse((source_lat,source_lon))
    target = geolocator.reverse((target_lat,target_lon))

    # Create a map that shows the correct public transport services to take from the source to the target
    m = folium.Map(location=[selected_path[0][0], selected_path[0][1]], zoom_start=13)

    # Add markers for the source and target points
    folium.Marker(location=[selected_path[0][0], selected_path[0][1]], popup="Origen: {}".format(source), icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(location=[selected_path[-1][0], selected_path[-1][1]], popup="Destino: {}".format(target), icon=folium.Icon(color='red')).add_to(m)

    # Add markers for the nearest stop from the source and target points
    source_coords = (selected_path[0][1], selected_path[0][0])
    near_source_stops, source_orientations = find_nearest_stops(source, margin)
    #print(near_source_stops)
    
    target_coords = (selected_path[-1][1], selected_path[-1][0])
    near_target_stops, target_orientations = find_nearest_stops(target, margin)
    
    pre_valid_services = []
    for source_stop_id in near_source_stops:
        for target_stop_id in near_target_stops:
            services = connection_finder(route_stops, source_stop_id, target_stop_id)
            pre_valid_services.extend(services)
    
    valid_services = []
    for service in pre_valid_services:
        for source_stop_id in near_source_stops:
            for target_stop_id in near_target_stops:
                source_orientation = get_bus_orientation(service, source_stop_id)
                target_orientation = get_bus_orientation(service, target_stop_id)
                if source_orientation == target_orientation:
                    valid_services.extend(service)
    
    if len(valid_services) == 0:
        print("There are no available services right now to go to the desired destination.")
        print("Please take into account that nighttime goes between 00:00:00 and 05:30:00.")
        return
    
    nighttime_flag = is_nighttime(source_hour)
    rush_hour_flag = is_rush_hour(source_hour)
    holiday_flag = is_holiday(source_date)
    if holiday_flag:
        rush_hour_flag = 0
    
    # Nighttime check
    daily_time_services = check_night_routes(valid_services, nighttime_flag)
    
    # Rush hour check
    valid_services = check_express_routes(daily_time_services, rush_hour_flag)

    valid_services = list(set(valid_services))
    
    valid_source_stops = [stop_id for stop_id in near_source_stops if any(route_id in valid_services for route_id in route_stops.keys() if stop_id in route_stops[route_id])]
    valid_source_stops = list(set(valid_source_stops))
    valid_target_stops = [stop_id for stop_id in near_target_stops if any(route_id in valid_services for route_id in route_stops.keys() if stop_id in route_stops[route_id])]
    valid_target_stops = list(set(valid_target_stops))

    # Give info
    print("")
    print("Routes have been found.")
    print("Calculating the best route and getting the arrival times for the next buses...")
    
    best_option = None
    best_option_times = None
    source_time = timedelta(hours=source_hour.hour, minutes=source_hour.minute, seconds=source_hour.second)
    
    valid_orientations = set(source_orientations)
    
    for stop_id in valid_source_stops:
        routes_at_stop = get_routes_at_stop(route_stops, stop_id)
        valid_stop_services = [stop_id for stop_id in valid_services if stop_id in routes_at_stop]
        for valid_service in valid_stop_services:
            a_t = get_arrival_times(valid_service, stop_id, source_date)

            if a_t is not None:
                arrival_times = a_t[1]
                time_until_next_buses = get_time_until_next_bus(arrival_times, source_hour)
                
                if not time_until_next_buses:
                    print("There are no available services right now to go to the desired destination.")
                    print("Please take into account that nighttime goes between 00:00:00 and 05:30:00.")
                    return

                # Print the time until the next three buses in the desired format
                for i in range(len(time_until_next_buses)):
                    minutes, seconds = time_until_next_buses[i]
                    waiting_time = timedelta(minutes=minutes, seconds=seconds)
                    arrival_time = source_time + waiting_time
                    time_string = timedelta_to_hhmm(arrival_time)

                    # Update the best option
                    if best_option is None or (arrival_time < best_option[2]):
                        best_option = (valid_service, stop_id, arrival_time, waiting_time)
                        best_option_times = time_until_next_buses

    # Print the best option
    print("")
    print("To go from: {}".format(source))
    print("To: {}".format(target))
    arrival_time = None
    if best_option is not None:
        best_arrival_time_str = timedelta_to_hhmm(best_option[2])
        print("The best option is to take the route {} on stop {}. The next bus arrives at {}.".format(best_option[0], best_option[1], best_arrival_time_str))
        print("The other two next buses arrives in:")
        for i in range(len(best_option_times)):
            if i == 0:
                continue
            minutes, seconds = best_option_times[i]
            waiting_time = timedelta(minutes=minutes, seconds=seconds)
            arrival_time = source_time + waiting_time
            time_string = timedelta_to_hhmm(arrival_time)
            print(f"{minutes} minutes, {seconds} seconds ({time_string})")
            
    
    # Map the options
    for stop_id in near_source_stops:
        #if stop_id not in valid_source_stops:
        #    stop_coords = get_stop_coords(route_stops, str(stop_id))
        #    folium.Marker(location=[stop_coords[1], stop_coords[0]], 
        #                  popup="Paradero {}, cercano al inicio.".format(stop_id), 
        #                  icon=folium.Icon(color='lightgray', icon='minus')).add_to(m)
        if stop_id in valid_source_stops:
            stop_coords = get_stop_coords(route_stops, str(stop_id))
            routes_at_stop = get_routes_at_stop(route_stops, stop_id)
            valid_stop_services = [stop_id for stop_id in valid_services if stop_id in routes_at_stop]

            for service in valid_stop_services:
                if service == best_option[0] and stop_id == best_option[1]:
                    folium.Marker(location=[stop_coords[1], stop_coords[0]], 
                          popup="Mejor opción: subirse al recorrido {} en el paradero {}.".format(best_option[0], best_option[1]), 
                          icon=folium.Icon(color='cadetblue', icon='plus')).add_to(m)
                    initial_distance = [(selected_path[0][0], selected_path[0][1]),(stop_coords[1], stop_coords[0])]
                    folium.PolyLine(initial_distance,color='black',dash_array='10').add_to(m)
                    
    for stop_id in near_target_stops:
        #if stop_id not in valid_target_stops:
        #    stop_coords = get_stop_coords(route_stops, str(stop_id))
        #    folium.Marker(location=[stop_coords[1], stop_coords[0]], 
        #                  popup="Paradero {}, cercano al destino.".format(stop_id), 
        #                  icon=folium.Icon(color='lightgray', icon='minus')).add_to(m)
        if stop_id in valid_target_stops:
            closest_stop = find_closest_stop((target_lat,target_lon), valid_target_stops)
            stop_coords = get_stop_coords(route_stops, str(stop_id))
            routes_at_stop = get_routes_at_stop(route_stops, stop_id)
            valid_stop_services = [stop_id for stop_id in valid_services if stop_id in routes_at_stop]
            
    for service in valid_stop_services:
        if service == best_option[0]:
            round_trip_id = service + "-I-" + get_trip_day_suffix(source_date)
            return_trip_id = service + "-R-" + get_trip_day_suffix(source_date)
            best_travel_time = None
            selected_stop = None
            for stop_id in valid_target_stops:
                round_time = get_travel_time(round_trip_id, [best_option[1], stop_id])
                return_time = get_travel_time(return_trip_id, [best_option[1], stop_id])
                
                if round_time is not None and round_time > timedelta() and (best_travel_time is None or round_time < best_travel_time):
                    best_travel_time = round_time
                    selected_stop = stop_id
                elif return_time is not None and return_time > timedelta() and (best_travel_time is None or return_time < best_travel_time):
                    best_travel_time = return_time
                    selected_stop = stop_id

            selected_stop_coords = get_stop_coords(route_stops, selected_stop)
            minutes, seconds = timedelta_separator(best_travel_time)
            
            print("")
            print("You will get off the bus on stop {} after {} minutes and {} seconds.".format(selected_stop, minutes, seconds))
            
            folium.Marker(location=[selected_stop_coords[1], selected_stop_coords[0]], 
                  popup="Mejor opción: bajarse del recorrido {} en el paradero {}.".format(best_option[0], selected_stop), 
                  icon=folium.Icon(color='cadetblue', icon='plus')).add_to(m)
            ending_distance = [(selected_path[-1][0], selected_path[-1][1]),(selected_stop_coords[1], selected_stop_coords[0])]
            folium.PolyLine(ending_distance,color='black',dash_array='10').add_to(m)
            
            total_time = best_option[3] + best_travel_time
            minutes, seconds = timedelta_separator(total_time)
            
            destination_time = source_time + total_time
            time_string = timedelta_to_hhmm(destination_time)
            print(f"Total travel time: {minutes} minutes, {seconds} seconds. You will arrive your destination at {time_string}.")
    
    # Set the optimal zoom level for the map
    fit_bounds(selected_path, m)

    return m


In [64]:
# ALGORITHM SCRIPT

In [65]:
# Calculate the distances between the given coordinates and the target point

stop_1 = get_stop_coords(route_stops, "PI1412")
stop_2 = get_stop_coords(route_stops, "PI1413")

distance1 = haversine(stop_1[0], stop_1[1], -70.76493, -33.48318)
distance2 = haversine(stop_2[0], stop_2[1], -70.76493, -33.48318)

# Compare the distances and determine the closest coordinate
if distance1 < distance2:
    closest_coordinate = "PI1412"
else:
    closest_coordinate = "PI1413"

print("The closest coordinate to (-33.48318, -70.76493) is:", closest_coordinate)

The closest coordinate to (-33.48318, -70.76493) is: PI1412


In [68]:
def connection_scan(graph, source_address, target_address, departure_time, departure_date):
    """
    The Connection Scan algorithm is applied to search for travel routes from the source to the destination,
    given a departure time and date. By default, the algorithm uses the current date and time of the system.
    However, you can specify a different date or time if needed.

    Args:
        graph (graph): the graph used to visualize the travel routes.
        source_address (string): the source address of the travel.
        target_address (string): the destination address of the travel.
        departure_time (time): the time at which the travel should start.
        departure_date (date): the date on which the travel should be done.

    Returns:
        list: the list of coordinates of the travel connections needed to arrive at the destination.
    """
    node_id_mapping = create_node_id_mapping(graph)
    
    source_node = address_locator(graph, source_address)
    target_node = address_locator(graph, target_address)

    if source_node is not None and target_node is not None:
        # Convert source and target node IDs to integers
        source_node_graph_id = graph.vertex_properties["graph_id"][source_node]
        target_node_graph_id = graph.vertex_properties["graph_id"][target_node]

        print("Both addresses have been found.")
        print("Processing...")
        #print("SOURCE NODE: {}. TARGET NODE: {}.".format(source_node_graph_id, target_node_graph_id))

        path = [source_node_graph_id, target_node_graph_id]
        path_coords = []
        for node in path:
            lon, lat = graph.vertex_properties["lon"][node], graph.vertex_properties["lat"][node]
            path_coords.append((lat, lon))
        #print(path)

        return path_coords
    else:
        return


def csa_commands():
    """
    Process the inputs given by the user to run the Connection Scan Algorithm.
    """

    # System's date and time
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    #print("Fecha y hora actuales =", dt_string)

    # Date formatting
    today = date.today()
    today_format = today.strftime("%d/%m/%Y")

    # Time formatting
    moment = now.strftime("%H:%M:%S")
    used_time = datetime.strptime(moment, "%H:%M:%S").time()

    # User inputs
    # Date and time
    source_date = input(
        "Enter the travel's date, in DD/MM/YYY format (press Enter to use today's date) : ") or today_format
    print(source_date)
    source_hour = input(
        "Enter the travel's start time, in HH:MM:SS format (press Enter to start now) : ") or used_time
    if source_hour != used_time:
        source_hour = datetime.strptime(source_hour, "%H:%M:%S").time()
    print(source_hour)
    ## MODIFICAR ESTO

    # Source address
    source_example = "Beauchef 850, Santiago"
    while True:
        source_address = input(
            "Enter the starting point's address, in 'Street #No, Province' format (Ex: 'Beauchef 850, Santiago'):") or source_example
        if source_address.strip() != '':
            #print("Dirección de Destino ingresada: " + target_address)
            break

    # Destination address
    destination_example = "Campus Antumapu Universidad de Chile, Santiago"
    while True:
        target_address = input(
            "Enter the ending point's address, in 'Street #No, Province' format (Ex: 'Campus Antumapu Universidad de Chile, Santiago'):")or destination_example
        if target_address.strip() != '':
            #print("Dirección de Destino ingresada: " + target_address)
            break

    path_coords = connection_scan(undirected_graph, source_address, target_address, source_hour, source_date)
    
    if path_coords:
        map = create_transport_map(route_stops, path_coords, source_date, source_hour, 0.15)
        if map:
            display(map)

        return path_coords

    else:
        print("")
        print("Something went wrong. Please try again later.")
        return

#avenida libertador bernardo o'higgins 5121, santiago
#avenida libertador bernardo o'higgins 1460, santiago

selected_path = csa_commands()

#TAREAS
#NECESITO QUE LOS RECORRIDOS DEL POPUP DE LOS PARADEROS VALIDOS SEAN SOLO LOS QUE PASAN POR DICHO PARADERO (LISTO)
#REVISAR HORARIOS DE RECORRIDOS EXPRESS Y AGREGARLOS AL CHECKEO DE HORARIO, 05:30 a 09:00 y 17:30 a 21:00 (LISTO)

#REVISAR COMO OBTENER "STOP_SEQUENCE" DESDE "STOP_TIMES" PARA OBTENER EL ORDEN DE LOS PARADEROS (LISTO)
#CON EL ORDEN, HACERLE SORT A LOS PARADEROS PARA PODER GRAFICAR LA RUTA COMO UNA LINEA COLOREADA DE FORMA CORRECTA
#(PARCIAL?)

#BUSCAR UNA MANERA DE REVISAR LOS DIAS DE FIN DE SEMANA EN PYTHON (LISTO)
#REVISAR COMO OBTENER LOS "CALENDAR_DATES" Y LOS FINES DE SEMANA PARA ELIMINAR LOS EXPRESS COMO VALIDOS (LISTO)

#REVISAR COMO OBTENER "ARRIVAL_TIME" DE "STOP_TIMES" PARA SABER A QUE HORA DEBIERA LLEGAR LA MICRO A UN PARADERO (LISTO)
#CON ESTO, UBICAR "SOURCE_HOUR" ENTRE LOS "ARRIVAL_TIMES" PARA OBTENER CUÁNTO HAY QUE ESPERAR LA MICRO (LISTO)
#ADEMÁS, CALCULAR LA DIFERENCIA ENTRE LOS "ARRIVAL_TIME" DEL PARADERO DE INICIO Y DE EL DE TERMINO PARA OBTENER EL TIEMPO DE VIAJE (LISTO)
#SUMANDO LOS TIEMPOS, SE PUEDE OBTENER EL TIEMPO COMPLETO DEL VIAJE Y APROXIMAR LA HORA DE LLEGADA (LISTO)

#REVISAR COMO OBTENER "DIRECTION_ID" DESDE "TRIPS" PARA OBTENER LA ORIENTACION DE LA MICRO (LISTO)

#REVISAR COMO HACER UN LOOP SOBRE LOS START_TIME Y END_TIME EN "FREQUENCIES.TXT" PARA REVISAR LOS RECORRIDOS 24 HORAS
#(UN RECORRIDO ES 24 HORAS SI SU PRIMER START_TIME ES 00:00:00 Y SU ULTIMO END_TIME ES 24:00:00)
#OBTENIENDO LOS RECORRIDOS 24 HORAS, HAY QUE AÑADIRLOS A LOS RECORRIDOS MOSTRADOS DE NOCHE
#EVENTUALMENTE HAY QUE MODIFICAR ESE CRITERIO
#CON LA ORIENTACION, FILTRAR LOS PARADEROS PARA QUE NO SOLO LA MICRO PASE, SINO QUE PASE CON LA MISMA ORIENTACION
#REVISAR CASOS DONDE LAS LISTAS SON VACIAS PARA QUE EL PROGRAMA NO MUERA
#CUANDO TODO FUNCIONE, ELIMINAR EL CHEQUEO EN "ADDRESS_LOCATOR" PARA USAR SOLO COORDENADAS DE NODOS DEL GRAFO DE OSM



Enter the travel's date, in DD/MM/YYY format (press Enter to use today's date) : 
13/07/2023
Enter the travel's start time, in HH:MM:SS format (press Enter to start now) : 
20:11:41
Enter the starting point's address, in 'Street #No, Province' format (Ex: 'Beauchef 850, Santiago'):calle laguna verde oriente 31 maipu
Enter the ending point's address, in 'Street #No, Province' format (Ex: 'Campus Antumapu Universidad de Chile, Santiago'):av el rosal 4578 maipu
Both addresses have been found.
Processing...


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [78]:

# TESTING AREA #


In [ ]:
def find_best_route(route_stops, selected_path):
    # Get the stops for each bus route
    route_stops_coords = {}
    for route_id, stops in route_stops.items():
        route_stops_coords[route_id] = [stop_info["coordinates"] for stop_info in stops.values()]

    # Calculate the average distance between each route's stops and the vertices on the selected path
    route_averages = {}
    for route_id, stops_coords in route_stops_coords.items():
        distances = []
        for path_coord in selected_path:
            path_lon, path_lat = path_coord
            stop_distances = []
            for stop_coords in stops_coords:
                stop_lon, stop_lat = stop_coords
                distance = haversine(path_lon, path_lat, stop_lon, stop_lat)
                stop_distances.append(distance)
            distances.append(min(stop_distances))
        route_averages[route_id] = sum(distances) / len(distances)

    # Find the route with the smallest average distance
    best_route_id, _ = min(route_averages.items(), key=lambda x: x[1])

    return best_route_id

print(best_route_id)

# Find the bus route that corresponds better to the set path
best_route_id = find_best_route(route_stops, selected_path)


# Map the stops of the best bus route
map = map_route_stops(best_route_id)
folium.Marker(location=selected_path[0], popup="SOURCE",
                       icon=folium.Icon(color='blue', icon='plus')).add_to(map)
folium.Marker(location=selected_path[-1], popup="TARGET",
                       icon=folium.Icon(color='blue', icon='plus')).add_to(map)
map